# Imports

In [1]:
import torch
import torch.nn as nn

from torchsummary import summary

# MobileNet-v1

In [2]:
class MobileNetV1(nn.Module):
    def __init__(self, in_channel, n_classes):
        super(MobileNetV1, self).__init__()

        def conv_bn(inp, outp, stride):
            """
            Convolutional Layer with Batch Normalization
            """
            return nn.Sequential(
                nn.Conv2d(inp, outp, 3, stride, 1, bias=False),
                nn.BatchNorm2d(outp, affine=False),
                nn.ReLU(inplace=True)
            )

        def conv_dsc(inp, outp, stride):
            """
            conv_dsc: Depthwise Separable Convolutional Layer
            groups = inp: Each input channel is convolved with its own set of filters
            """
            return nn.Sequential(
                # Depthwise Layer
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp, affine=False),
                nn.ReLU(inplace=True),

                # Pointwise Layer
                nn.Conv2d(inp, outp, 1, 1, 0, bias=False),
                nn.BatchNorm2d(outp, affine=False),
                nn.ReLU(inplace=True),
            )

        # MobileNet Body Architecture
        self.model = nn.Sequential(
            conv_bn(in_channel, 8, 2),
            conv_dsc(8, 16, 1),
            conv_dsc(16, 32, 2),
            conv_dsc(32, 32, 1),
            conv_dsc(32, 64, 2),
            conv_dsc(64, 64, 1),
            conv_dsc(64, 128, 2),
            conv_dsc(128, 128, 1),
            conv_dsc(128, 128, 1),
            conv_dsc(128, 128, 1),
            conv_dsc(128, 128, 1),
            conv_dsc(128, 128, 1),
            conv_dsc(128, 256, 2),
            conv_dsc(256, 256, 2)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [4]:
model = MobileNetV1(3, 1)
summary(model, input_size=(3, 448, 448), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 224, 224]             216
       BatchNorm2d-2          [-1, 8, 224, 224]               0
              ReLU-3          [-1, 8, 224, 224]               0
            Conv2d-4          [-1, 8, 224, 224]              72
       BatchNorm2d-5          [-1, 8, 224, 224]               0
              ReLU-6          [-1, 8, 224, 224]               0
            Conv2d-7         [-1, 16, 224, 224]             128
       BatchNorm2d-8         [-1, 16, 224, 224]               0
              ReLU-9         [-1, 16, 224, 224]               0
           Conv2d-10         [-1, 16, 112, 112]             144
      BatchNorm2d-11         [-1, 16, 112, 112]               0
             ReLU-12         [-1, 16, 112, 112]               0
           Conv2d-13         [-1, 32, 112, 112]             512
      BatchNorm2d-14         [-1, 32, 1